# Trading Algorithm Template

This template shows you how to develop a trading algorithm with Quantiacs. 
* You can run it by clicking **Shift+Enter** for each notebook cell. 
* You can implement your own algorithm in the **strategy** function definition. 
* Once you are satisfied with the results you can click the **Submit to the contest** tag. Your system will be evaluated live and take part to the Quantiacs contests.

In [1]:
%%javascript
window.IPython && (IPython.OutputArea.prototype._should_scroll = function(lines) { return false; })
// run this cell for disabling widget scrolling

<IPython.core.display.Javascript object>

This template implements a simple **moving-average crossover** system.

It uses the Quantiacs built-in backtester which performs a **multi-pass** simulation.

The multi-pass approach isolates each point in time (hence the name multi-pass) and for each point in time it processes the algorithm using only past data. It is slower than a vectorized **single-pass** approach where at any moment the full time series is available to the algorithm. However, the multi-pass approach is more reliable as results are not artificially inflated by unintentional forward-looking on sequential data.

Once the following steps are clear, you can implement your own algorithm inside the **strategy** function defined below. 

Let us first import the necessary modules:

In [2]:
import xarray as xr

We use **xarray** for data manipulation. Basic informations can be found on our [**documentation**](https://quantiacs.com/documentation/en/user_guide/xarray.html) page or on the xarray [**official**](https://docs.xarray.dev/en/stable/) page.

If you prefer to work with [**pandas**](https://pandas.pydata.org/) data structures, simply convert two-dimensional xarrray data structures to pandas ones. 

Let us consider a concrete example. You start loading Quantiacs data. 
```python
data = qndata.load_ndx_data(tail=365*5)
```

These are delivered as a three-dimensional data structure indexed by (time, asset, field). Basic fields are the open, close, high and low daily prices. **Close** prices can be selected using:

```python
close = data.sel(field="close")
```

The result is a two-dimensional data structure indexed by (time, asset). You can convert this two-dimensional xarray data structure  for the **close price** to a two-dimensional pandas data structure using:

```python
import pandas
close_p = close.to_pandas()
```

After defining your strategy and computing allocation weights, let us say called weights_p (pandas data structure), you can convert them to an xarray data structure (time, asset) for the final allocation **weights** as follows:

```python
weights = weights_p.unstack().to_xarray()
```

Next we import the fundamental modules of the Quantiacs library. You can inspect the source code in the **qnt** directory in your root folder. Or on our [**github**](https://github.com/quantiacs) page.

In [3]:
import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library

In the next cell we define the function loading market data:

In [4]:
def load_data(period):
    """This function loads the data. In the Documentation you will find more details
    on the different datasets you can load. You can inspect the source code in your
    root folder.
    """
    
    # loads NASDAQ-100 stock data for the Q18 stock contest:
    return qndata.stocks.load_ndx_data(tail = period, dims = ("time", "field", "asset"))

The next cell contains the strategy definition. It must return **allocation weights**: the fraction of capital you want to allocate to each asset on a daily basis. You can take short positions by using negative weights.

In [5]:
def strategy(data):
    """This function contains your strategy. It is a simple moving-average crossover system.
    It must return allocation weights for all assets for the next trading day (FIXED point in time (note isel(time=-1))) 
    based on the input xarray "data" (data for lookback period).
    """
    
    close = data.sel(field="close") #select closing price
    
    ma_slow = qnta.sma(close, 200).isel(time=-1)
    ma_fast = qnta.sma(close, 20).isel(time=-1)
    
    # keep a long position on the asset when the fast moving average of the price
    # is larger than the slow one:
    
    weights = xr.where(ma_slow < ma_fast, 1, -1) # 1 - long position (positive exposure), 
                                                 #-1 - short position (negative exposure)
    
    # this field tags stocks which, at a given point in time,
    # are included in the NASDAQ-100 index:
    
    is_liquid = data.sel(field="is_liquid")

    # filter weights so that we are exposed only to stocks which, at a given point in time,
    # are or were part of the NASDAQ-100 index, to avoid survivorship bias:
    
    weights = weights * is_liquid
    
    # here you can normalize the weights. If the sum of the absolute values for the weights is
    # larger than 1, Quantiacs rescale them down uniformly so that the sum of the absolute
    # values for the weights is equal to 1.
    
    weights = weights / 100.0

    return weights

The final call computes the weights. Check carefully **ERROR** and **WARNINGS**, control plots and if you are happy with the results, click the **Submit to the contest** button!

In [6]:
# Here you can run the backtester which evaluates the performance of your strategy since start_date
# lookback_period is expressed in calendar days and it expresses the length
# of the rolling window used by the backtester. It must be large enough
# to include all indicators for your strategy (expressed in trading days).
# The smaller the window, the more efficient the evaluation.

weights = qnbt.backtest(
    competition_type = "stocks_nasdaq100",
    load_data        = load_data,
    lookback_period  = 365*4,
    start_date       = "2006-01-01",
    strategy         = strategy
)

Run last pass...
Load data...


100% (367973 of 367973) |################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (46708 of 46708) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (13140364 of 13140364) |############| Elapsed Time: 0:00:00 Time:  0:00:000:00


fetched chunk 1/2 4s


100% (5515124 of 5515124) |##############| Elapsed Time: 0:00:00 Time:  0:00:000:00


fetched chunk 2/2 5s
Data loaded 5s
Run strategy...
Load data for cleanup...


...

---
Align...
Calc global stats...
---
Calc stats per asset...


/root/book/qnt/stats.py:792: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',) The recommendation is to set join explicitly for this case.



Build plots...
---
Select the asset (or leave blank to display the overall stats):


/root/book/qnt/stats.py:792: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',) The recommendation is to set join explicitly for this case.



interactive(children=(Combobox(value='', description='asset', options=('', 'NAS:AAL', 'NAS:AAPL', 'NAS:ABNB', …

For improving speed you can implement your strategy using a **single-pass approach**. Your algorithm must return arrays for the allocation weights. In this implementation the full time series is accessible to your algo at any point in time, so you should make sure that no forward looking is taking place and your algo does not use future information for predicting the past. 

**IMPORTANT: Any** implementation in your Notebook (single-pass or multi-pass) will be processed **after submission on our servers** using a multi-pass approach in order to prevent forward looking. 

If the **Sharpe ratio** of your submission does not match your expectations from your Notebook research, please review your implementation: most likely some forward looking is taking place, for example by computing some global mean value which is then used for taking trading decisions.

```python
# Single-pass implementation (for prototyping)
import xarray as xr

import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.stats as qns

# load data:
data = qndata.stocks.load_ndx_data(min_date="2006-01-01")

# calculate weights:
close = data.sel(field="close")
ma_slow = qnta.sma(close, 200)
ma_fast = qnta.sma(close, 20)
weights = xr.where(ma_fast > ma_slow, 1, -1)

# liquidity filter:
is_liquid = data.sel(field="is_liquid")

# set weights:
weights = weights * is_liquid
weights = weights / 100.0

# clean weights taking corner cases into account:
weights = qnout.clean(weights, data, "stocks_nasdaq100")

# check before submission:
qnout.check(weights, data, "stocks_nasdaq100")

# write results:
qnout.write(weights)

# calculate statistics for checking:
stats = qns.calc_stat(data, weights.sel(time=slice("2006-01-01",None)))
stats.to_pandas().tail()
```